
> PCA: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html


- using SVD method. (Not eigenvalues & eigenvectors)
- input data is centered, but not scaled beforing applying SVD

> TruncatedSVD: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

- do NOT centeralize
- altorithoms: a fast randomized solver, or the naive method (enigenvector of A^TA)

> np.linalg.svd https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html

U, s, Vt = np.linalg.svd(X)

In [41]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [207]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.datasets import load_iris

In [208]:
iris_dataset = load_iris()

In [209]:
X = iris_dataset.data
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

## PCA()

In [250]:
m0 = PCA(n_components=None)
m0.fit(X)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [253]:
m0.components_   # Vt

array([[ 0.36138659, -0.08452251,  0.85667061,  0.3582892 ],
       [ 0.65658877,  0.73016143, -0.17337266, -0.07548102],
       [-0.58202985,  0.59791083,  0.07623608,  0.54583143],
       [-0.31548719,  0.3197231 ,  0.47983899, -0.75365743]])

In [255]:
Z0 = m.transform(X)

In [256]:
m0.explained_variance_
m0.explained_variance_ratio_

array([4.22824171, 0.24267075, 0.0782095 , 0.02383509])

array([0.92461872, 0.05306648, 0.01710261, 0.00521218])

In [257]:
m0.singular_values_

array([25.09996044,  6.01314738,  3.41368064,  1.88452351])

Definition of Frobenius Norm:

$$ \|A\|_F = \Big(\sum_i \sum_j (a_{ij})^2 \Big)^{\frac{1}{2}} $$


Lemma 15.1:  if svd of $A$ is $(U, \Sigma, V^T)$, then
$$ \|A\|_F = \sqrt{\sigma_1^2 + \sigma_2^2 + \cdots + \sigma_n^2} $$


Theorem 15.2: （原书中的定理表述比较绕，以下是我的理解）

考虑最小化问题， 找矩阵A在 rank=k 矩阵空间中的、F范数意义下的最优近似：

$$ \min_{S \in \mathcal{M}} \| A - S \|_F $$

上述最小化问题的解存在

$$\exists S^* = \arg \min_{S \in \mathcal{M}} \| A - S \|_F $$

且可以通过截断奇异值分解的方式得到


$$ S^* = U \Sigma^\prime V^T $$

where
$$ \Sigma^\prime = diag(\sigma_1, \cdots, \sigma_k) $$
$$ A = U \Sigma V^T $$

In PCA ---------------------------------------------------------------

Objective: minimize Projection Error

//TODO: how do "min ProjectionError" and "max Var" related ?
总体、样本 主成分定义

In PCA ---------------------------------------------------------------

$$ ExplainedVarianceRatio = \frac{\| X_{approx, k} \|_F}{\| X \|_F} = \frac{\lambda_1 + \cdots + \lambda_k}{\sum_{i=1}^n \lambda_i}$$ 

or

$$ 1- ExplainedVarianceRatio = \frac{\| X - X_{approx, k} \|_F}{\| X \|_F} = 1 - \frac{\lambda_1 + \cdots + \lambda_k}{\sum_{i=1}^n \lambda_i} $$

In [467]:
k = 3

# centeralized, MUST
iris_dataset = load_iris()
X = iris_dataset.data
X = X - X.mean(axis=0)


m = PCA(n_components=k)
m.fit(X)
Z = m.transform(X)
X_approx = m.inverse_transform(Z)

def FrobeniusNorm(A):
    return np.sqrt(np.sum(A**2))

(FrobeniusNorm(X_approx) / FrobeniusNorm(X))**2
sum(m.explained_variance_ratio_)                   # equal  (X need to be centralized)

(FrobeniusNorm(X - X_approx) / FrobeniusNorm(X))**2
1 - sum(m.explained_variance_ratio_)                   # equal  (X need to be centralized)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

0.9947878161267255

0.9947878161267247

0.005212183873275374

0.005212183873275267

In [468]:
# |X|_F = sqrt(sum sigular_values^2)

FrobeniusNorm(X)
np.sqrt(np.sum(m0.singular_values_ ** 2))  # equal

FrobeniusNorm(X_approx)
np.sqrt(np.sum(m.singular_values_ ** 2))   # equal

26.103076447039722

26.103076447039726

26.034960555894003

26.034960555893996

In [469]:
# m.components_  is the Vt of 
#      U,S,Vt = svd(1/sqrt(m-1) X)
# transform X to Z:   Z = Vt dot X
# X_approx = V dot Z

Vt = m.components_
_Z = (Vt @ X.T).T
_X_approx = (Vt.T @ Z.T).T

np.allclose(_Z, Z)                 # True
np.allclose(_X_approx, X_approx)  # True

True

True

## Manually PCA

In [473]:
# data: centeralized
iris_dataset = load_iris()
X = iris_dataset.data
X = X - X.mean(axis=0)


m, n = X.shape
X_prime = 1 / np.sqrt(m - 1) * X
U, s, Vt = np.linalg.svd(X_prime)

S = np.zeros_like(X)
np.fill_diagonal(S, s)

In [474]:
k = 3

ZZ = (Vt[:k] @ X.T).T

ZZ[:5]
Z[:5]   # 注意，第二、三主成分 符号相反。这个应该是正常的

np.allclose(np.abs(ZZ), np.abs(Z))

array([[-2.68412563, -0.31939725,  0.02791483],
       [-2.71414169,  0.17700123,  0.21046427],
       [-2.88899057,  0.14494943, -0.01790026],
       [-2.74534286,  0.31829898, -0.03155937],
       [-2.72871654, -0.32675451, -0.09007924]])

array([[-2.68412563,  0.31939725, -0.02791483],
       [-2.71414169, -0.17700123, -0.21046427],
       [-2.88899057, -0.14494943,  0.01790026],
       [-2.74534286, -0.31829898,  0.03155937],
       [-2.72871654,  0.32675451,  0.09007924]])

True

In [495]:
XX_approx = (Vt[:k].T @ ZZ.T).T

np.allclose(XX_approx, X_approx)   # inverse_transformation 能恢复到原始的X

True

In [496]:
(FrobeniusNorm(X - XX_approx) / FrobeniusNorm(X)) ** 2

0.005212183873275376

In [497]:
(s ** 2) / np.sum(s ** 2)

array([0.92461872, 0.05306648, 0.01710261, 0.00521218])

## np.linalg.svd

In [387]:
U, s, Vh = np.linalg.svd(X)

In [388]:
U.shape
s.shape
Vh.shape

(150, 150)

(4,)

(4, 4)

In [395]:
S = np.zeros_like(X)
np.fill_diagonal(S, s)

S.shape

(150, 4)

In [397]:
np.allclose(U @ S @ Vh, X)

True

## numpy notes

### @ operator 

In [336]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[100, 0, 0], [0, 300, 0]])

In [335]:
A @ B    # dot product

array([[ 100,  600,    0],
       [ 300, 1200,    0]])

In [332]:
np.dot(A, B)

array([[ 100,  600,    0],
       [ 300, 1200,    0]])

In [333]:
A.dot(B)

array([[ 100,  600,    0],
       [ 300, 1200,    0]])

In [329]:
A * A   # element-wise

array([[ 1,  4],
       [ 9, 16]])

In [331]:
np.multiply(A, A)   # element-wise

array([[ 1,  4],
       [ 9, 16]])

### slicing with None: np.newaxis

https://stackoverflow.com/questions/1408311/numpy-array-slice-using-none

None: just alias of np.newaxis

In [436]:
x = np.array([1,2,3])
x

array([1, 2, 3])

In [428]:
x[:, np.newaxis]

array([[1],
       [2],
       [3]])

In [429]:
x[:, None]

array([[1],
       [2],
       [3]])

In [435]:
x[None, :]

array([[1, 2, 3]])

### `...` literal

https://stackoverflow.com/questions/42190783/what-does-three-dots-in-python-mean-when-indexing-what-looks-like-a-number

In [358]:
x = np.array([1,2,3])
x

array([1, 2, 3])

In [359]:
x[..., None]

array([[1],
       [2],
       [3]])

In [362]:
x[None, ...]

array([[1, 2, 3]])

In [418]:
?Ellipsis

Type:        ellipsis
String form: Ellipsis
Namespace:   Python builtin
Docstring:   <no docstring>


In [415]:
x=[]
x.append(x)
x

[[...]]

Type:        ellipsis
String form: Ellipsis
Namespace:   Python builtin
Docstring:   <no docstring>
